In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
## DO NOT DELETE ##

def generate_linear_data(n:int, d:int, sigma:float):
  '''
    Input:
      n: number of datapoints
      d: dimension of features
      sigma: std for gaussian noise for the synthetic linear data
    returns 3 variables
      X the design matrix shape is (n x d) each element in the matrix is in [0, 1]
      y the associated labels shape is (n,) each y is in [-1, 1]
      theta is randomly generated on the surface of the sphere,
  '''
  X = np.random.randn(n, d) # entries uniform in [0,1), thus each row ||x_i|| bounded by 1
  theta = np.random.normal(0, 1, d) # d iid Gaussian each entry N(0, 1)
  theta = theta / np.linalg.norm(theta) # theta is uniformly distributed on the surface of unit sphere, shape (d,)
  y = (X @ theta) + np.random.normal(0, sigma, size=n) # so essentially the learner is trying to recover \theta
  return X, y

In [ ]:
## DO NOT DELETE ##
def mse_loss(y_true:np.ndarray, y_pred:np.ndarray):
  '''
  Inputs:
    y_true : the ground truth labels vector
    y_pred : the predicted labels vector
  '''
  assert len(y_true) == len(y_pred) # assert that the size is same

  return mean_squared_error(y_true, y_pred) # MSE calculation

In [ ]:
def linear_regression_soln(X_train:np.ndarray, y_train:np.ndarray, \
                           X_test:np.ndarray, y_test:np.ndarray) -> np.ndarray:
    '''
     TODO complete this function ;
     Implement linear regression and compute the mse (3 PTS)

     Returns:
        mse_loss_linear_reg (float) : the mean squared error on the test set
    '''

    # TODO Implement this function

    # return mse_loss_linear_reg

In [ ]:
def ridge_regression_soln(X_train:np.ndarray, y_train:np.ndarray, \
                        X_test:np.ndarray, y_test:np.ndarray, \
                        lamb:float) -> np.ndarray:
    '''
     TODO complete this function ;
     Implement ridge regression and compute the mse (3 PTS)

    HINT --- Solve this in three steps

    Step (1) -- Compute the optimal theta vector on the train set (call it `theta_opt`)
    This is a (np.ndarray) of size d (d = number of features.

    Step (2) -- Compute the prediction vector using the above on the test set and return this prediction vector.

    Step (3) -- Compute MSE (see the corresponding function in utils.py file) and return the MSE value

    Returns:
        mse_loss_ridge_reg (float) : the mean squared error on the test set
    '''

    # TODO Implement this function

    # return mse_loss_ridge_reg

In [ ]:
## DO NOT CHANGE THE BELOW LINES !

X, y = generate_linear_data(n = 1000, d = 20, sigma = 0.0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 37)
# N_train, N_test = len(X_train), len(X_test)

mse_loss_linear_reg = linear_regression_soln(X_train, y_train, X_test, y_test)
print(mse_loss_linear_reg) # -- Sanity check; this value should be very close to 0, think why!

lambda_list = [0.1, 0.5, 1, 5, 10]
mse_list = []

for lamb in lambda_list:

    mse_loss_ridge_reg = ridge_regression_soln(X_train, y_train, X_test, y_test, lamb)
    mse_list.append(mse_loss_ridge_reg)

print(mse_list)

In [ ]:
# TODO plot the mse values (`mse_list`) on the y-axis and the lambda values on the x-axis for ridge regression
# use matplotlib